In [1]:
import pandas as pd
import ast

In [2]:
from text_matcher import matcher


In [4]:
pairs=pd.read_csv("../Downloads/Code - Knowledge Graph/Code - Knowledge Graph/pairs.csv")

In [5]:
def conv(inp):
    return ast.literal_eval(inp)

In [6]:
pairs["neighbors"]=pairs["neighbors"].apply(conv)
pairs["non-neighbors"]=pairs["non-neighbors"].apply(conv)

In [7]:
tp=set()
fp=set()
for i,row in pairs.iterrows():
    p1=row[2]
    for n in row[3]:
        
        pair=[p1,n]
        pair.sort()
        tp.add(tuple(pair))
    for n in row[4]:
        pair=[p1,n]
        pair.sort()
        fp.add(tuple(pair))

In [8]:
len(tp)

9751

In [9]:
len(fp)

22266

In [10]:
path="C:/Users/lucia/Downloads/Code - Knowledge Graph/Code - Knowledge Graph/semantic/"

In [18]:
%%capture

is_cite=[]
p1=[]
p2=[]
score=[]
lens=[]
failed=0
count=0
for p in tp:
    try:
        if count%50==0:
            print(count)
        count+=1
        
        ta = matcher.Text(open(path+p[0]+".txt", encoding="utf-8").read()[350:-2000], 'p1')
        tb = matcher.Text(open(path+p[1]+".txt", encoding="utf-8").read()[350:-2000], 'p2')
        score.append(matcher.Matcher(ta, tb).match()[0])
        lens.append(len(ta.text)+len(tb.text))
        p1.append(p[0])
        p2.append(p[1])
        is_cite.append(1)
    except:
        failed+=1
        continue

In [19]:
%%capture
count=0

for p in fp:
    try:
        if count>15000:
            break
        count+=1
        
        ta = matcher.Text(open(path+p[0]+".txt", encoding="utf-8").read()[350:-2000], 'p1')
        tb = matcher.Text(open(path+p[1]+".txt", encoding="utf-8").read()[350:-2000], 'p2')
        score.append(matcher.Matcher(ta, tb).match()[0])
        lens.append(len(ta.text)+len(tb.text))
        p1.append(p[0])
        p2.append(p[1])
        is_cite.append(0)
    except:
        failed+=1
        continue

In [20]:
out=pd.DataFrame([p1,p2,is_cite,lens,score]).T
out.to_csv("textreuse.csv")

In [21]:
out[4]=out[4]/out[3]

In [22]:
out[out[2]==1][4].mean()

4.424911882236665e-06

In [23]:
out[out[2]==0][4].mean()

2.815995701034936e-06